# Some basic things to remember in Transfer Learning :

In [1]:
import tensorflow as tf

## Remeber model imported from keras and tensorflow are totally different in use, Though there function and weights are same but if using one only one of the library/Framework must be used :
### Looking at two models both ResNet_50 but there layer names are very different :
1. Looking at Keras Model :

In [5]:
# model summary in keras : 
from tensorflow import keras
from keras.applications.resnet50 import ResNet50
model = ResNet50(include_top = False,
                     weights = 'imagenet',
                     input_shape = (150,150,3))
model.summary()

C:\Users\shaur\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 150, 150, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 75, 75, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

2. Looking at tensorflow's keras model

In [6]:
# model summary in tensorflow :
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(include_top = False,
                     weights = 'imagenet',
                     input_shape = (150,150,3))
model.summary()

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 4d473c1dd8becc155b73f8504c6f6626 so we will re-download the data.
94773248/94765736 [==============================] - 24s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
___________________________

It can seen that both the models have same number of trainable and non-trainable parameters but the output layer names are very different.   
Also every model has some Non-trainable parameters by default so if we do :
````
model.trainable = False 
````
The above line turns all the parameters frozen .  
Now if we load inception model the layer names are all same so if we use either of frameworks we won't get an error.  
observe :


In [13]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model = InceptionV3(include_top = False,
                     weights = 'imagenet',
                     input_shape = (150,150,3))
model.trainable = False
# observe the trainable parameters now as they go to zero
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [19]:
# now using keras to make a model to flatten the layers : 
from tensorflow import keras 
model_keras = keras.Sequential([
    model,
    keras.layers.Flatten(),
    keras.layers.Dense(2,activation = 'sigmoid')
])
# but the summary of the model gives only the model see :
model_keras.summary()

Now if we do the same as above with ResNet50 model it won't work as either of the framework won't able to identify the other.  
Let's see with the example : 

In [30]:
from tensorflow.keras.applications.resnet50 import ResNet50
model = ResNet50(include_top = False,
                     weights = 'imagenet',
                     input_shape = (150,150,3))
import keras 
model_keras = keras.Sequential([
    model,
    keras.layers.Flatten(),
    keras.layers.Dense(2,activation = 'sigmoid')
])

TypeError: The added layer must be an instance of class Layer. Found: <tensorflow.python.keras.engine.training.Model object at 0x000001D306E44CC8>

but if we use the tensorflow's sequential it will work :

In [31]:
model_keras = tf.keras.Sequential([
    model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2,activation = 'sigmoid')
])
model_keras.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_6 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102402    
Total params: 23,690,114
Trainable params: 23,636,994
Non-trainable params: 53,120
_________________________________________________________________


#### if we turn the trainable's to False the paramters in ``model_keras`` will also get all it's parameters associated with ResNet as 0  

In [32]:
model.trainable = False
model_keras.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 5, 5, 2048)        23587712  
_________________________________________________________________
flatten_6 (Flatten)          (None, 51200)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 102402    
Total params: 23,690,114
Trainable params: 102,402
Non-trainable params: 23,587,712
_________________________________________________________________


we can see that all associated parameters with model i.e. ResNet have turned down to 0 thus parameters are freezed

### Moving the model to a specific layer name 

In [23]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model = InceptionV3(include_top = False,
                     weights = 'imagenet',
                     input_shape = (150,150,3))
model.trainable = False
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 74, 74, 32)   864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 74, 74, 32)   96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 74, 74, 32)   0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [27]:
# moving model to a specific layer : 
output_layer = model.get_layer('mixed9')
output_layer_value = output_layer.output

model_keras = tf.keras.Model(model.input,output_layer_value)
model_keras.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 74, 74, 32)   864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 74, 74, 32)   96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 74, 74, 32)   0           batch_normalization_188[0][0]    
______________________________________________________________________________________________

Thus the above code gets the layer moved to a layer with name `mixed9` as we observed in the summary of inception   
Now changing the `model`'s value again reflects the same effects in `model_keras`

In [29]:
model.trainable = False
model_keras.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 74, 74, 32)   864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 74, 74, 32)   96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 74, 74, 32)   0           batch_normalization_188[0][0]    
______________________________________________________________________________________________

## Now looking at all the tow different ways to form models : 
### 1. Using Sequential to make the model as a class in sequential class : 

In [36]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
base_model = MobileNetV2(input_shape = (150,150,3),
                        include_top = False,
                        weights = 'imagenet')
base_model.summary()

C:\Users\shaur\anaconda3\lib\site-packages\keras_applications\mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 19s 2us/step
Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________

__________________________________________________________________________________________________
block_6_pad (ZeroPadding2D)     (None, 21, 21, 192)  0           block_6_expand_relu[0][0]        
__________________________________________________________________________________________________
block_6_depthwise (DepthwiseCon (None, 10, 10, 192)  1728        block_6_pad[0][0]                
__________________________________________________________________________________________________
block_6_depthwise_BN (BatchNorm (None, 10, 10, 192)  768         block_6_depthwise[0][0]          
__________________________________________________________________________________________________
block_6_depthwise_relu (ReLU)   (None, 10, 10, 192)  0           block_6_depthwise_BN[0][0]       
__________________________________________________________________________________________________
block_6_project (Conv2D)        (None, 10, 10, 64)   12288       block_6_depthwise_relu[0][0]     
__________

In [38]:
# taking 'block_16_project_BN' as the last layer :
last_layer = base_model.get_layer('block_16_project_BN')
model = tf.keras.Model(base_model.input,last_layer.output)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [39]:
base_model.trainable = False
model.summary() # this shows the model always depends on the base_model 

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_8[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [41]:
# now forming a model with sequential : 
# model.trainable = True we can use this to again make the model trainable

model_keras = tf.keras.Sequential([
    model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])
model_keras.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 5, 5, 320)         1843264   
_________________________________________________________________
flatten_7 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              8193024   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total params: 10,037,313
Trainable params: 10,005,761
Non-trainable params: 31,552
_________________________________________________________________


In [45]:
model.trainable = False # or base_model.trainable = False gives us model with only our added parameters
model_keras.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 5, 5, 320)         1843264   
_________________________________________________________________
flatten_7 (Flatten)          (None, 8000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              8193024   
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total params: 10,037,313
Trainable params: 8,194,049
Non-trainable params: 1,843,264
_________________________________________________________________


### 2. Without Sequential : 

In [46]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
base_model = MobileNetV2(input_shape = (150,150,3),
                        include_top = False,
                        weights = 'imagenet')

base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [50]:
# considering the same last layer to compare parameters
last_layer = base_model.get_layer('block_16_project_BN')

X = tf.keras.layers.Flatten()(last_layer.output)
X = tf.keras.layers.Dense(1024,activation = 'relu')(X)
X = tf.keras.layers.Dense(1,activation = 'sigmoid')(X)

model = tf.keras.Model(base_model.input,X)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

In [51]:
# we see the abbove parameters match 
# using base_model to change trainable para :
base_model.trainable = False
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 151, 151, 3)  0           input_9[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 75, 75, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 75, 75, 32)   128         Conv1[0][0]                      
____________________________________________________________________________________________

## thus we observed both the ways and saw how the things changed if training in base_model was altered